In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from os import listdir
from PIL import Image
import json
import random
import math

In [2]:
class YOLOv1(nn.Module):
    """
    This class contains the YOLOv1 model. It consists of 24 convolutional and
    2 fully-connected layers which divide the input image into a 
    split_size x split_size grid and predict num_boxes bounding boxes per grid
    cell. If the confidence of a bounding box reaches a certain value, it is 
    considered as a valid prediction.
    """
    
    def __init__(self, split_size, num_boxes, num_classes):
        """
        Initializes the neural-net with the parameter values to produce the
        desired predictions.
        
        Parameters:
            split_size (int): Size of the grid which is applied to the image.
            num_boxes (int): Amount of bounding boxes which are predicted per 
            grid cell.
            num_classes (int): Amount of different classes which are being 
            predicted by the model.
        """
        
        super(YOLOv1, self).__init__()
        self.darkNet = nn.Sequential(
            nn.Conv2d(3, 64, 7, padding=3, stride=2, bias=False), # 3,448,448 -> 64,224,224
                nn.BatchNorm2d(64),
                nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2, 2), # -> 64,112,112
            
            nn.Conv2d(64, 192, 3, padding=1, bias=False), # -> 192,112,112
                nn.BatchNorm2d(192),
                nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2, 2), # -> 192,56,56
            
            nn.Conv2d(192, 128, 1, bias=False), # -> 192,56,56
                nn.BatchNorm2d(128),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 256, 3, padding=1, bias=False), # -> 256,56,56
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 256, 1, bias=False), # -> 256,56,56
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, 3, padding=1, bias=False), # -> 512,56,56
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2, 2), # -> 512,28,28
            
            nn.Conv2d(512, 256, 1, bias=False), # -> 256,28,28
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, 3, padding=1, bias=False), # -> 512,28,28
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 256, 1, bias=False), # -> 256,28,28
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, 3, padding=1, bias=False), # -> 512,28,28
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 256, 1, bias=False), # -> 256,28,28
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, 3, padding=1, bias=False), # -> 512,28,28
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 256, 1, bias=False), # -> 256,28,28
                nn.BatchNorm2d(256),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(256, 512, 3, padding=1, bias=False), # -> 512,28,28
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 512, 1, bias=False), # -> 512,28,28
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False), # -> 1024,28,28
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(2, 2), # -> 1024,14,14
            
            nn.Conv2d(1024, 512, 1, bias=False), # -> 512,14,14
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False), # -> 1024,14,14
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 512, 1, bias=False), # -> 512,14,14
                nn.BatchNorm2d(512),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(512, 1024, 3, padding=1, bias=False), # -> 1024,14,14
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False), # -> 1024,14,14
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 1024, 3, padding=1, stride=2, bias=False), # -> 1024,7,7
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False), # -> 1024,7,7
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(1024, 1024, 3, padding=1, bias=False), # -> 1024,7,7
                nn.BatchNorm2d(1024),
                nn.LeakyReLU(0.1, inplace=True),
            )
        self.fc = nn.Sequential(
            nn.Linear(1024 * split_size * split_size, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Linear(4096, split_size * split_size * (num_classes + num_boxes*5)),
            )
        
    def forward(self, x):
        """
        Forwards the input tensor through the model to produce the predictions. 
        
        Parameters:
            x (tensor): A tensor of shape (batch_size, 3, 448, 448) which represents
            a batch of input images.
                
        Returns:
            x (tensor): A tensor of shape (batch_size, split_size * split_size * (num_classes
            + num_boxes*5)) which contains the predicted bounding boxes.
        """
        
        x = self.darkNet(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [3]:
class YOLO_Loss():
    """
    Used to calculate the loss for the YOLO-model using a batch of predictions and the corresponding
    ground-truth labels. 
    """
    
    def __init__(self, predictions, targets, split_size, num_boxes, num_classes, lambda_coord, lambda_noobj):
        """
        Initialize the parameters for calculating the loss value.
        
        Parameters:
            predictions (tensor): A tensor containing a mini-batch of training samples.
            targets (tensor): A tensor containing a mini-batch of ground-truth labels.
            split_size (int): Specifies the size of the grid which is applied to the image.
            num_boxes (int): Amount of bounding boxes which are predicted by the YOLO-model.
            num_classes (int): Amount of classes which are being predicted.
            lambda_cooord (float): Hyperparameter for the loss regarding the bounding box coordinates.
            lambda_noobj (float): Hyperparameter for the loss in case there is no object in the cell.
        """
        
        self.predictions = predictions
        self.targets = targets
        self.split_size = split_size
        self.cell_dim = int(448 / split_size) # Dimension of a single cell
        self.num_boxes = num_boxes
        self.num_classes = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        self.final_loss = 0 # Here will the final value of the loss be stored
        
        
    def loss(self):
        """
        Main function for calculating the loss. Stores the calculated loss inside the final_loss atribute.
        """
        
        for sample in range(self.predictions.shape[0]):
            mid_loss = 0 # Loss of the centre coordinates
            dim_loss = 0 # Loss of the width and height values
            conf_loss = 0 # Loss of the confidence score when there is an object in the cell
            conf_loss_noobj = 0 # Loss of the confidence score when there is no object in the cell
            class_loss = 0 # Loss of the class score
            for cell_h in range(split_size):
                for cell_w in range(split_size):
                    # Check if the current cell contains an object
                    if self.targets[sample, 0, cell_h, cell_w] != 1:
                        conf_loss_noobj += self.noobj_loss(sample, cell_h, cell_w)
                    else:
                        mid_loss_local, dim_loss_local, conf_loss_local, class_loss_local = self.obj_loss(sample, cell_h, cell_w)
                        mid_loss += mid_loss_local
                        dim_loss += dim_loss_local
                        conf_loss += conf_loss_local
                        class_loss += class_loss_local
                        
            # Calculate the final loss by summing the other losses and applying the hyperparameters lambda_coord and lambda_noobj
            self.final_loss += lambda_coord*mid_loss + lambda_coord*dim_loss + lambda_noobj*conf_loss_noobj + conf_loss + class_loss
                        
                    
    def noobj_loss(self, sample, cell_h, cell_w):
        """
        Calculates the loss value for a single cell in case there is no ground-truth object in that cell.
        
        Parameters:
            sample (int): The index of the current sample from the batch.
            cell_h (int): Index of the cell coordinate.
            cell_w (int): Index of the cell coordinate.
            
        Return:
            loss_value (float): The value of the loss with respect to the cell.
        """
        
        """ Version which only penalizes the best bounding box
        best_box = 0 # Used to store the index of the best bounding box
        max_conf = 0 # Used to store the highest bounding box confidence score
        for box in range(self.num_boxes):
            box_conf = self.predictions[sample, box*5, cell_h, cell_w]
            if box_conf > max_conf:
                max_conf = box_conf
                best_box = box # Store the box index with the highest confidence
        # Use the box with the highest confidence score for the conf_loss_noobj
        loss_value = (0 - self.predictions[sample, best_box*5, cell_h, cell_w])**2
        return loss_value
        """
        
        loss_value = 0
        for box in range(self.num_boxes):
            loss_value += (0 - self.predictions[sample, box*5, cell_h, cell_w])**2
        
        return loss_value

        
    def obj_loss(self, sample, cell_h, cell_w): 
        """
        Calculates the loss value for a single cell in case there is a ground-truth object in that cell.
        
        Parameters:
            sample (int): The index of the current sample from the batch.
            cell_h (int): Index of the cell coordinate.
            cell_w (int): Index of the cell coordinate.
            
        Return:
            mid_loss_local (float): Loss value for the mid coordinates of the bounding box.
            dim_loss_local (float): Loss value for the height and width coordinates of the bounding box.
            conf_loss_local (float): Loss value for the confidence score.
            class_loss_local (float): Loss value for the class scores.
        """
        
        # Finds the box with the highest IoU and stores its index in best_box
        best_box = self.find_best_box(sample, cell_h, cell_w)
        
        # Calculates the loss for the centre coordinates
        x_loss = (self.targets[sample, 1, cell_h, cell_w] - self.predictions[sample, 1+best_box*5, cell_h, cell_w])**2        
        y_loss = (self.targets[sample, 2, cell_h, cell_w] - self.predictions[sample, 2+best_box*5, cell_h, cell_w])**2
        mid_loss_local = x_loss + y_loss
                
        # Calculates the loss for the width and height values
        w_loss = (math.sqrt(self.targets[sample, 3, cell_h, cell_w]) - math.sqrt(self.predictions[sample, 3+best_box*5, cell_h, cell_w]))**2
        h_loss = (math.sqrt(self.targets[sample, 4, cell_h, cell_w]) - math.sqrt(self.predictions[sample, 4+best_box*5, cell_h, cell_w]))**2
        dim_loss_local = w_loss + h_loss
                
        # Calculates the loss of the confidence score
        conf_loss_local = (1 - self.predictions[sample, best_box*5, cell_h, cell_w])**2
                
        # Calculates the loss for the class scores
        class_loss_local = 0
        for c in range(self.num_classes):
            class_loss_local += (self.targets[sample, 5+c, cell_h, cell_w] - self.predictions[sample, 5*num_boxes+c, cell_h, cell_w])**2
            
        return mid_loss_local, dim_loss_local, conf_loss_local, class_loss_local
                        
    
    def find_best_box(self, sample, cell_h, cell_w):
        """
        Finds the bounding box with the highest IoU with respect to the ground-truth bounding box.
        
        Parameters:
            sample (int): The index of the current sample from the batch.
            cell_h (int): Index of the cell coordinate.
            cell_w (int): Index of the cell coordinate.
            
        Returns:
            best_box (int): The index of the bounding box with the highest IoU.
        """
        
        best_box = 0
        max_iou = 0
        
        # Transform the box coordinates into the corner format
        t_box_coords = self.MidtoCorner(self.targets[sample, 1:5, cell_h, cell_w], cell_h, cell_w)
        
        for box in range(self.num_boxes):
            # Transform the box coordinates into the corner format
            p_box_coords = self.MidtoCorner(self.predictions[sample, 1+box*5:5+box*5, cell_h, cell_w], cell_h, cell_w)
                    
            box_score = self.IoU(t_box_coords, p_box_coords)
            if box_score > max_iou:
                max_iou = box_score
                best_box = box # Store the box order with the highest IoU                
        return best_box
        
        
    def MidtoCorner(self, mid_box, cell_h, cell_w):
        """
        Transforms bounding box coordinates which are in the mid YOLO format into the common corner format
        with the correct pixel distance.
        
        Parameters:
            mid_box (list): Bounding box coordinates which are in the mid YOLO format.
            cell_h (int): Height index of the cell with the bounding box.
            cell_w (int): Width index of the cell with the bounding box.
            
        Returns:
            corner_box (list): A list containing the coordinates of the bounding box in the common
            corner foormat
        """
        
        # Transform the coordinates from the YOLO format into normal pixel values
        centre_x = mid_box[0]*self.cell_dim + self.cell_dim*cell_w
        centre_y = mid_box[1]*self.cell_dim + self.cell_dim*cell_h
        width = mid_box[2] * 448
        height = mid_box[3] * 448

        # Calculate the corner values of the bounding box
        x1 = int(centre_x - width/2)
        y1 = int(centre_y - height/2)
        x2 = int(centre_x + width/2)
        y2 = int(centre_y + height/2)

        corner_box = [x1,y1,x2,y2]  
        return corner_box    
    
    
    def IoU(self, target, prediction):
        """
        Calculates the Intersection over Union of two bounding boxes.
        
        Parameters:
            target (list): A list with bounding box coordinates in the corner format.
            predictions (list): A list with bounding box coordinates in the mid format.
            
        Returns:
            iou_value (float): The score of the IoU over the two boxes
        """
        # Calculate the corner coordinates of the intersection
        i_x1 = max(target[0], prediction[0])
        i_y1 = max(target[1], prediction[1])
        i_x2 = min(target[2], prediction[2])
        i_y2 = min(target[3], prediction[3])

        intersection = max(0,(i_x2-i_x1)) * max(0,(i_y2-i_y1))    
        union = ((target[2]-target[0]) * (target[3]-target[1])) + ((prediction[2]-prediction[0]) * (prediction[3]-prediction[1])) - intersection

        iou_value = intersection / union    
        return iou_value

In [4]:
class DataLoader():
    """
    This class uses its attributes to load the training data and transforms it into tensors.
    The tensors are then stored in mini-batches inside the train_data list which is the final 
    product of this class. Multiple function calls of LoadData() will initialize the train_data 
    list with new tensors from the training data, excluding all the previous ones. 
    """

    def __init__(self, train_files_path, target_files_path, category_list, split_size, batch_size, train_size):
        """
        Initialize all parameters for loading and transforming the data into tensors.
        
        Parameters:
            train_files_path (string): The path to the train image folder
            target_files_path (string): The path to the json file containg the image labels
            category_list (list): Reference list to all the label categories for object detection
            split_size (int): Amount of grid cells
            batch_size (int): Batch size
            train_size (int): Amount of images which are loaded as training data 
        """
        
        self.train_files_path = train_files_path
        self.target_files_path = target_files_path       
        self.category_list = category_list        
        self.num_classes = len(category_list)       
        self.cells = split_size        
        self.batch_size = batch_size      
        self.train_size = train_size
        
        self.train_files = [] # Will contain the remaining image names from the folder
        self.target_files = [] # Will contain the json elements with the ground-truth labels
        
        self.train_data = [] # Will contain tuples with mini-batches of image and label tensors    
        self.img_tensors = [] # Used to temporary store samples from a single batch
        self.target_tensors = [] # Used to temporary store samples from a single batch
        
        # Define transform which is applied to every single image to resize and convert it into a tensor
        self.transform = transforms.Compose([
            transforms.Resize((448,448), Image.NEAREST),
            transforms.ToTensor(),
            ])
    

    def LoadFiles(self):
        """
        First function to be executed.
        Loads the images and the label file using the respective system path.
        """
            
        # All image names from the directory are loaded into the list train_files.
        self.train_files = listdir(self.train_files_path)
        
        # The json file containing the labels is loaded into the list target_files.
        f = open(self.target_files_path)
        self.target_files = json.load(f)
        
        
    def LoadData(self):
        """
        Transforms the training images and labels into tensors and loads them into batches. Once a batch is
        full, it is stored in the train_data list. Fills the train_data list with batches until the desired
        train_size is reached. Every image that is loaded, is being excluded from future calls of this 
        function.
        """
        
        # Reset the cache
        self.train_data = []    
        self.img_tensors = [] 
        self.target_tensors = [] 

        for i in range(len(self.train_files)):
            # Check if batch is full and perhaps start a new one
            if len(self.img_tensors) >= self.batch_size:
                self.train_data.append((torch.stack(self.img_tensors), torch.stack(self.target_tensors)))
                self.img_tensors = []
                self.target_tensors = []
                print('Loaded batch ', len(self.train_data), 'of ', int(self.train_size/self.batch_size))
                print('Percentage Done: ', round(len(self.train_data)/int(self.train_size/self.batch_size)*100., 2), '%')
                print('')
            
            if i == self.train_size:
                break # The train_data list is full with the desired amount of batches
                
            # Extracts a single random image and the corresponding label, and transforms them into
            # tensors. Both are appended to the img_tensors and target_tensors lists
            self.extract_image_and_label() 


    def extract_image_and_label(self):
        """
        Chooses a random image which is then being transformed into a tensor and stored.
        Finds the corresponding label inside the json file which is then being transformed into a tensor
        and stored. Stores both tensors inside the img_tensors and target_tensors lists.
        """
        
        img_tensor, chosen_image = self.extract_image()
        target_tensor = self.extract_json_label(chosen_image)

        self.img_tensors.append(img_tensor)
        self.target_tensors.append(target_tensor)

        
    def extract_image(self):   
        """
        Finds a random image from the train_files list and applies the transform to it. 
 
        Returns:
            img_tensor (tensor): The tensor which contains the image values
            f (string): The string name of the image file
        """    
        
        f = random.choice(self.train_files)
        self.train_files.remove(f)
        global img
        img = Image.open(self.train_files_path + f)
        img_tensor = self.transform(img) # Apply the transform to the image.
        return img_tensor, f


    def extract_json_label(self, chosen_image):
        """
        Uses the name of the image to find the corresponding json element. Then it extracts the data and
        transforms it into a tensor which is stored inside the target_tensors list.

        Parameters:
            chosen_image (string): The name of the image for which the label is needed.

        Returns:
            target_tensor (tensor): The tensor which contains the image labels
        """
        
        for json in self.target_files:
            if json['name'] == chosen_image:
                img_label = json
                target_tensor = self.transform_label_to_tensor(img_label)
                return target_tensor
                #break

        #target_tensor = self.transform_label_to_tensor(img_label)
        #return target_tensor


    def transform_label_to_tensor(self, img_label):
        """
        Extracts the useful information from the json element and transforms them into a tensor.
        
        Parameters:
            img_label (): A specific json element
            
        Returns:
            target_tensor (tensor): A tensor of size (5+num_classes,cells,cells) which is used as the target of 
            the image.
        """
        
        target_tensor = torch.zeros(5+self.num_classes, self.cells, self.cells) # Here are the information stored

        for labels in range(len(img_label["labels"])):

            # Store the category index if its contained within the category_list.
            category = img_label["labels"][labels]["category"]         
            if category not in self.category_list:
                continue
            ctg_idx = self.category_list.index(category)

            # Store the bounding box information and rescale it by the resize factor.
            x1 = img_label["labels"][labels]["box2d"]["x1"] * (448/img.size[0])
            y1 = img_label["labels"][labels]["box2d"]["y1"] * (448/img.size[1])
            x2 = img_label["labels"][labels]["box2d"]["x2"] * (448/img.size[0])
            y2 = img_label["labels"][labels]["box2d"]["y2"] * (448/img.size[1])

            # Transforms the corner bounding box information into a mid bounding box information
            x_mid = abs(x2 - x1) / 2 + x1
            y_mid = abs(y2 - y1) / 2 + y1
            width = abs(x2 - x1) 
            height = abs(y2 - y1) 

            # Size of a single cell
            cell_dim = int(448 / self.cells)

            # Determines the cell position of the bounding box
            cell_pos_x = int(x_mid // cell_dim)
            cell_pos_y = int(y_mid // cell_dim)

            # Stores the information inside the target_tensor
            if target_tensor[0][cell_pos_y][cell_pos_x] == 1: # Check if the cell already contains an object
                continue
            target_tensor[0][cell_pos_y][cell_pos_x] = 1
            target_tensor[1][cell_pos_y][cell_pos_x] = (x_mid % cell_dim) / cell_dim
            target_tensor[2][cell_pos_y][cell_pos_x] = (y_mid % cell_dim) / cell_dim
            target_tensor[3][cell_pos_y][cell_pos_x] = width / 448
            target_tensor[4][cell_pos_y][cell_pos_x] = height / 448
            target_tensor[ctg_idx+5][cell_pos_y][cell_pos_x] = 1

        return target_tensor

In [5]:
def TrainNetwork(num_epochs):
    """
    Starts the training process of the model.
    Parameters:
        num_epochs (int): Amount of epochs for training the model
    """
    
    data = DataLoader(train_files_path, target_files_path, category_list, split_size, batch_size, train_size)
    
    for epoch in range(num_epochs):
        model.train()
        
        print("DATA IS BEING LOADED FOR A NEW EPOCH")
        print("")
        data.LoadFiles()
        
        if len(data.train_files) % 1000 == 0:
            print("Saving checkpoint")
            print("")
            torch.save(model.state_dict(), check_point_path)
        
        while len(data.train_files) > 0:
            print("LOADING NEW BATCHES")            
            print("Remaining files:" + str(len(data.train_files)))
            print("")
            data.LoadData()
            
            for batch_idx, (train_data, target_data) in enumerate(data.train_data):
                train_data = train_data.to(device)
                target_data = target_data.to(device)
    
                predictions = model(train_data)
                predictions = predictions.view(batch_size, num_boxes*5 + num_classes, split_size, split_size)
                yolo_loss = YOLO_Loss(predictions, target_data, split_size, num_boxes, num_classes, lambda_coord, lambda_noobj)
                yolo_loss.loss()
                loss = yolo_loss.final_loss
                
                optimizer.zero_grad()
                loss.backward()

                optimizer.step()

                print('Train Epoch: {} of {} [Batch: {}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                    epoch+1, num_epochs, batch_idx+1, len(data.train_data),
                    (batch_idx+1) / len(data.train_data) * 100., loss))
                print('')

                
# Dataset parameters
train_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k/images/100k/train/"
target_files_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/bdd100k_labels_release/bdd100k/labels/det_v2_train_release.json"
check_point_path = "C:/Users/alens/Desktop/Real-time-Object-Detection-for-Autonomous-Driving-using-Deep-Learning/YOLO v1/YOLOv1 implementation/Training Checkpoint/checkpoint.pth"
category_list = ["other vehicle", "pedestrian", "traffic light", "traffic sign", "truck", "train", "other person", "bus", "car", "rider", "motorcycle", "bicycle", "trailer"]


# Hyperparameters
learning_rate = 0.0001
split_size = 7
num_boxes = 1
num_classes = len(category_list)
lambda_coord = 5
lambda_noobj = 0.5
batch_size = 10
num_epochs = 10
train_size = 100


# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

# Initialize model
model = YOLOv1(split_size, num_boxes, num_classes).to(device)

# Define the learning method as stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Start the training process
TrainNetwork(num_epochs)

DATA IS BEING LOADED FOR A NEW EPOCH

Saving checkpoint

LOADING NEW BATCHES
Remaining files:70000

Loaded batch  1 of  10
Percentage Done:  10.0 %

Loaded batch  2 of  10
Percentage Done:  20.0 %

Loaded batch  3 of  10
Percentage Done:  30.0 %

Loaded batch  4 of  10
Percentage Done:  40.0 %

Loaded batch  5 of  10
Percentage Done:  50.0 %

Loaded batch  6 of  10
Percentage Done:  60.0 %

Loaded batch  7 of  10
Percentage Done:  70.0 %

Loaded batch  8 of  10
Percentage Done:  80.0 %

Loaded batch  9 of  10
Percentage Done:  90.0 %

Loaded batch  10 of  10
Percentage Done:  100.0 %

Train Epoch: 1 of 10 [Batch: 1/10 (10%)] Loss: 537.724243

Train Epoch: 1 of 10 [Batch: 2/10 (20%)] Loss: 572.196594

Train Epoch: 1 of 10 [Batch: 3/10 (30%)] Loss: 502.331604

Train Epoch: 1 of 10 [Batch: 4/10 (40%)] Loss: 473.335388

Train Epoch: 1 of 10 [Batch: 5/10 (50%)] Loss: 484.312164

Train Epoch: 1 of 10 [Batch: 6/10 (60%)] Loss: 440.314667

Train Epoch: 1 of 10 [Batch: 7/10 (70%)] Loss: 334.178

TypeError: expected Tensor as element 8 in argument 0, but got NoneType